In [1]:
import os
import torch

cache_dir = "/Users/yangye/models"
# 设置环境变量
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 检查环境变量是否设置成功
print(os.environ["HF_ENDPOINT"])
# 设置 CUDA_VISIBLE_DEVICES 环境变量
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



# if torch.cuda.is_available():
#     print(f"Number of GPUs available: {torch.cuda.device_count()}")
#     print("CUDA version:", torch.version.cuda)
#     print("Device name:", torch.cuda.get_device_name(0))
# else:
#     print("CUDA is not available. Check your installation and try again."

https://hf-mirror.com


In [5]:
from transformers import BertTokenizer, DistilBertTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
sentences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this.",
]
tokens = [tokenizer.tokenize(sentence) for sentence in sentences]
ids = [tokenizer.convert_tokens_to_ids(token) for token in tokens]
print("tokens:", tokens)
print("ids:", ids)


tokens: [['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.'], ['i', 'hate', 'this', '.']]
ids: [[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012], [1045, 5223, 2023, 1012]]


In [6]:
# 报错，，张量维度不匹配，必须是矩形
# input_ids = torch.tensor(ids)

ValueError: expected sequence of length 14 at dim 1 (got 4)

In [7]:
ids = [[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012],
       [1045, 5223, 2023, 1012, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
input_ids = torch.tensor(ids)  

In [9]:
from transformers import AutoModelForSequenceClassification

ids1=torch.tensor([[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]])
ids2=torch.tensor([[1045, 5223, 2023, 1012]])

all_ids=input_ids

model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
print(model(ids1).logits)
print(model(ids2).logits)
print(model(all_ids).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)
tensor([[ 3.9497, -3.1357]], grad_fn=<AddmmBackward0>)
tensor([[-2.7276,  2.8789],
        [ 1.5444, -1.3998]], grad_fn=<AddmmBackward0>)


In [10]:
attention_mask=torch.tensor(
    [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
     [1, 1, 1, 1 , 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
)

In [13]:
print(model(ids1).logits)
print(model(ids2).logits)
print(model(all_ids,attention_mask=attention_mask).logits)

tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)
tensor([[ 3.9497, -3.1357]], grad_fn=<AddmmBackward0>)
tensor([[-2.7276,  2.8789],
        [ 3.9497, -3.1357]], grad_fn=<AddmmBackward0>)


In [14]:
tokenizer(sentences,padding=True)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 1045, 5223, 2023, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [16]:
from transformers import AutoTokenizer

checkpoint ="distilbert-base-uncased-finetuned-sst-2-english"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

sequence="I've been waiting for a HuggingFace course my whole life."

model_inputs=tokenizer(sequence)

print(model_inputs)


{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [20]:
sequences=[
    "I've been waiting for a HuggingFace course my whole life.",
    "So have I."
]

model_inputs=tokenizer(sequences,padding=True)

print(model_inputs)
print(model_inputs["input_ids"][0])
print(model_inputs["input_ids"][1])

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[101, 2061, 2031, 1045, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [22]:
model_inputs1=tokenizer(sequences,padding="max_length",max_length=8,truncation=True)
print(model_inputs1["input_ids"][0])
print(model_inputs1["input_ids"][1])


[101, 1045, 1005, 2310, 2042, 3403, 2005, 102]
[101, 2061, 2031, 1045, 1012, 102, 0, 0]


In [23]:
model_inputs2=tokenizer(sequences,padding="longest")
print(model_inputs2["input_ids"][0])
print(model_inputs2["input_ids"][1])

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[101, 2061, 2031, 1045, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [25]:
model_inputs3=tokenizer(sequences,padding="max_length")
print(model_inputs3["input_ids"][0])
print(model_inputs3["input_ids"][1])

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [30]:
model_inputs=tokenizer(sequences,truncation=True,max_length=8)
print(model_inputs)
print(model_inputs["input_ids"][0])
print(model_inputs["input_ids"][1])

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 102], [101, 2061, 2031, 1045, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}
[101, 1045, 1005, 2310, 2042, 3403, 2005, 102]
[101, 2061, 2031, 1045, 1012, 102]


In [34]:
model_inputs=tokenizer(sequences,return_tensors="pt",padding=True)
print(model_inputs["input_ids"][0])
print(model_inputs["input_ids"][1])


tensor([  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
         2607,  2026,  2878,  2166,  1012,   102])
tensor([ 101, 2061, 2031, 1045, 1012,  102,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])


In [36]:
model_inputs=tokenizer(sequences,return_tensors="pt",truncation=True)
print(model_inputs["input_ids"][0])
print(model_inputs["input_ids"][1])


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).